This notebook is used to support the ORKG-related statements made in the paper
<br><br>
<center>
"Examining the ORKG towards Representation of Control Theoretic Knowledge – Preliminary Experiences and Conclusions"

</center>
    
(currently under review).

Note: every SPARQL-Query can also be run via the web interface <https://www.orkg.org/orkg/sparql/>.

In [42]:
import time
print(time.ctime())
%load_ext ipydex.displaytools

Tue Feb 22 14:53:29 2022


In [50]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

sparql = SPARQLWrapper("https://www.orkg.org/orkg/triplestore/")
sparql.setReturnFormat(JSON)

prefixes = """
PREFIX orkgr: <http://orkg.org/orkg/resource/>
PREFIX orkgc: <http://orkg.org/orkg/class/>
PREFIX orkgp: <http://orkg.org/orkg/predicate/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

"""

**Claim:**  At the time of writing, the ORKG SPARQL query service lists
101218 distinct nodes (excluding literal values) and 4699 properties.


In [45]:
qs = """
SELECT  (COUNT(distinct ?vertex) AS ?count)
WHERE
{
  {
    ?vertex ?p []
  }
  UNION
  {
    [] ?p ?vertex
    FILTER(!IsLiteral(?vertex))
  }
}
"""

sparql.setQuery(qs)
ans = sparql.query().convert()
q = ans["results"]["bindings"] ##:

q := [{'count': {'type': 'typed-literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
   'value': '101218'}}]

---

In [44]:
qs = """
SELECT  (COUNT(distinct ?p) AS ?count)
WHERE
{
  {
    ?vertex ?p []
  }
  UNION
  {
    [] ?p ?vertex
    FILTER(!IsLiteral(?vertex))
  }
}
"""

sparql.setQuery(qs)
ans = sparql.query().convert()
q = ans["results"]["bindings"] ##:

q := [{'count': {'type': 'typed-literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
   'value': '4699'}}]

---

---

**Claim:**  At the time of writing, the ORKG SPARQL query service lists
101218 distinct nodes (excluding literal values) and 4699 properties.


In [ ]:
with open("dump.nt") as thefile:
    lines = thefile.readlines()
    
print(len(lines))
new_lines = []
for l in lines:
    if "<null>" in l:
        continue
    new_lines.append(l)

---

**Claim:** At the beginning of this study (2022-01-15) there are five papers
listed under research field Control Theory (R109).


In [60]:
qs = prefixes + """
select distinct ?paper ?title where
  {?paper orkgp:P30 orkgr:R109;
          rdfs:label ?title;
          rdf:type ?type.
  MINUS { ?paper rdf:type orkgc:PaperDeleted . }
  }
"""

sparql.setQuery(qs)
ans = sparql.query().convert()
q = ans["results"]["bindings"]

In [61]:
flat_result_list = [{k: v["value"] for k, v in row.items()} for row in q]
pd.DataFrame(flat_result_list)

,paper,title
0,http://orkg.org/orkg/resource/R138527,Sensitivity analysis applied to a variational ...
1,http://orkg.org/orkg/resource/R138532,Incremental projection approach of regularizat...
2,http://orkg.org/orkg/resource/R146945,Design and In Silico Evaluation of a Closed-Lo...
3,http://orkg.org/orkg/resource/R146974,Data-Enabled Predictive Control: In the Shallo...
4,http://orkg.org/orkg/resource/R146978,Data-Driven Model Predictive Control With Stab...
